What I learned so far:
- IOTP in Germany behaves strange. Throwing MQTT errno104 connection reset by peer. No other region does that
- paho mqtt client allocated memory for each message ~~published~~ queued
- ~~paho mqtt client has to be disconnected in order to free memory~~ a message has to actually leave the client to free up memory
- sending too many messages(amount varies) in a short time ~~will~~ may cause the iotp to disconnect you
- this may result in a client caught in some never ending loop
- the topic is always part of the message. 
- #IOTP will count the bytes in the message not the payload

## Dependencies

In [2]:
!pip install cloudant

## Credentials

In [3]:
credentials = {
  'username':'',
  'password':"",
  'host':'',
  'port':'443',
  'url':''
}

In [4]:
# The code was removed by DSX for sharing.

## Util functions

In [5]:
from cloudant import Cloudant
def getDB():
    database = "iotp-cost-calculator"
    client = Cloudant(credentials['username'], credentials['password'], url=credentials['url'], connect=True)
    return client[database]

In [6]:
import numpy
def removeOutliers(input):
    median = numpy.median(numpy.array(input))
    result = [s for s in input if s < 1.5*median and s > median/1.5]
    return result

## Number crunching

### Static overhead calculation

In [8]:
import numpy as np
db = getDB()
documents = db.get_view_result('_design/analysisEnriched', 'analysisEnriched',raw_result=True)['rows']

overall_static_overhead_list = {}
overall_static_overhead_list[0] = []
overall_static_overhead_list[1] = []
overall_static_overhead_list[2] = []
overall_percentage_list = {}
overall_percentage_list[1] = []
overall_percentage_list[2] = []

for actual_size in [58,113,527,1058,5323]:
    static_overhead_list = []
    qos_zero_mean = 0
    for qos in range(0,3):
        if(actual_size == 58 and qos ==2): # Filter out this one -> useless data
            continue
        percentage_diff_list = []
        for doc in documents:
            #print(doc)
            if doc['id'] != "_design/analysisEnriched" and doc['value']['qos'] == qos and doc['value']['actual_size'] == actual_size:
                percentage_diff_list.append(doc['value']['percentage_diff'])
        cleanedList = removeOutliers(percentage_diff_list)
        mean = np.mean(cleanedList)
        static_overhead = (actual_size*mean/100)-actual_size
        static_overhead_list.append(static_overhead)
        print("Qos: {}, actual size: {}".format(qos,actual_size))
        #print("Difference in data usages in percent:")
        #print(percentage_diff_list)
        #print("Cleaned:")
        #print(cleanedList)
        print("Cleaned Mean usage diff: {}%".format(mean))
        #print("Standard Deviation: {}".format(np.std(cleanedList)))
        print("Calculated static overhead: {}".format(static_overhead))
        overall_static_overhead_list[qos].append(static_overhead)
        if qos == 0:
            qos_zero_mean = mean
        if qos > 0:
            overall_percentage_list[qos].append(mean-qos_zero_mean)
    cleaned_static_overhead_list = removeOutliers(static_overhead_list)
    static_overhead_mean = np.mean(cleaned_static_overhead_list)
    #print(static_overhead_list)
    #print(cleaned_static_overhead_list)
    print("Mean static overhead for actual_size {}: {}".format(actual_size,static_overhead_mean))
    print("######################################################")
for qos in range(0,3):
    cleaned_static_overhead_list = {}
    cleaned_static_overhead_list[qos] = removeOutliers(overall_static_overhead_list[qos])
    static_overhead_mean = np.mean(cleaned_static_overhead_list[qos])
    print("Overall static overhead for qos {}: {}".format(qos,static_overhead_mean))
    
    if qos > 0:
        cleaned_percentage_list = {}
        cleaned_percentage_list[qos] = removeOutliers(overall_percentage_list[qos])
        percentage_mean = np.mean(cleaned_percentage_list[qos])
        print("Overall percentage mean for qos {}: {}".format(qos,percentage_mean))

Qos: 0, actual size: 58
Cleaned Mean: 217.956264138%
Calculated static overhead: 68.4146332
Qos: 1, actual size: 58
Cleaned Mean: 228.959875%
Calculated static overhead: 74.7967275
Mean static overhead for actual_size 58: 71.60568035
######################################################
Qos: 0, actual size: 113
Cleaned Mean: 162.281235398%
Calculated static overhead: 70.377796
Qos: 1, actual size: 113
Cleaned Mean: 182.885357522%
Calculated static overhead: 93.660454
Qos: 2, actual size: 113
Cleaned Mean: 211.21319823%
Calculated static overhead: 125.670914
Mean static overhead for actual_size 113: 96.5697213333
######################################################
Qos: 0, actual size: 527
Cleaned Mean: 113.177984364%
Calculated static overhead: 69.4479776
Qos: 1, actual size: 527
Cleaned Mean: 114.678289374%
Calculated static overhead: 77.354585
Qos: 2, actual size: 527
Cleaned Mean: 116.312523719%
Calculated static overhead: 85.967
Mean static overhead for actual_size 527: 77.58985

### Difference between qos data usage

In [15]:
import numpy as np
db = getDB()
documents = db.get_view_result('_design/analysisEnriched', 'analysisEnriched',raw_result=True)['rows']

overall_reported_list = {}
overall_reported_list[0] = []
overall_reported_list[1] = []
overall_reported_list[2] = []
for actual_size in [58,113,527,1058,5323]:
    for qos in range(0,3):
        reported_count = 0
        for doc in documents:
            #print(doc)
            if doc['id'] != "_design/analysisEnriched" and doc['value']['qos'] == qos and doc['value']['actual_size'] == actual_size:
                reported = doc['value']['reported_data_usage']
                reported_count += reported
        overall_reported_list[qos].append(reported_count)
        print("Qos: {}, overall reported: {}MB".format(qos,reported_count/1000000))
i=0
for actual_size in [58,113,527,1058,5323]:
    qos_diff_1 = overall_reported_list[1][i] - overall_reported_list[0][i]
    qos_diff_2 = overall_reported_list[2][i] - overall_reported_list[0][i]
    print("actual_size: {} qos_diff_1: {}MB qos_diff_2: {}MB".format(actual_size,qos_diff_1/1000000,qos_diff_2/1000000))
    i += 1

Qos: 0, overall reported: 154MB
Qos: 1, overall reported: 5312MB
Qos: 2, overall reported: 7581MB
Qos: 0, overall reported: 9072MB
Qos: 1, overall reported: 123646MB
Qos: 2, overall reported: 5168MB
Qos: 0, overall reported: 4809MB
Qos: 1, overall reported: 4243MB
Qos: 2, overall reported: 6521MB
Qos: 0, overall reported: 2607MB
Qos: 1, overall reported: 14747MB
Qos: 2, overall reported: 105791MB
Qos: 0, overall reported: 6746MB
Qos: 1, overall reported: 36771MB
Qos: 2, overall reported: 301MB
actual_size: 58 qos_diff_1: 5157MB qos_diff_2: 7426MB
actual_size: 113 qos_diff_1: 114573MB qos_diff_2: -3905MB
actual_size: 527 qos_diff_1: -566MB qos_diff_2: 1711MB
actual_size: 1058 qos_diff_1: 12139MB qos_diff_2: 103184MB
actual_size: 5323 qos_diff_1: 30025MB qos_diff_2: -6445MB


## Visualization

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
db = getDB()

for actual_size in [58,113,527,1058,5323]:
    print("Actual size: {}".format(actual_size))
    for qos in range(0,3):
        assumed_list = []
        reported_list = []
        timestamp_list = []
        sending_time_list = []
        data_usage_diff_list = []
        time_took_list = []
        percentage_diff_list = []
        for doc in db:
            #print(doc)
            if doc['_id'] != "_design/total_delta" and doc['qos'] == qos and doc['actual_size'] == actual_size:
                assumed_list.append(doc['assumed_delta_data_usage'])
                reported_list.append(doc['delta_data_usage'])
                data_usage_diff_list.append(doc['delta_data_usage'] - doc['assumed_delta_data_usage'])
                timestamp_list.append(doc['storage_timestamp'])
                time_took_list.append(doc['time_took'])
                sending_time_list.append(doc['sending_time'])
                percentage_diff_list.append(doc['delta_data_usage']*100.0 / (doc['assumed_delta_data_usage']*1.0))
        assumed_plot = plt.scatter(sending_time_list,assumed_list,c="green",label="assumed_delta_data_usage")
        reported_plot = plt.scatter(sending_time_list,reported_list,c="red",label="reported_delta_data_usage")
        data_usage_diff_plot = plt.scatter(sending_time_list,data_usage_diff_list,c="blue",label="data_usage_diff")
        plt.xlabel("sending_time")
        plt.ylabel("data_usage")
        plt.legend((assumed_plot,reported_plot,data_usage_diff_plot),("assumed_plot","reported_plot","data_usage_diff_plot"),bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
        plt.xscale('log')
        plt.yscale('log')
        plt.show()
        if len(data_usage_diff_list) > 0:    
            mean_delta = sum(data_usage_diff_list)/len(data_usage_diff_list)
        else:
            mean_delta = 0
        print("Qos: {}, actual size: {}".format(qos,actual_size))
        print("Mean delta: {}MB".format(mean_delta/1000000.0))
        print("Reported in MB:")
        print(map(lambda x: x/1000000.0,reported_list))
        print("Assumed in MB:")
        print(map(lambda x: x/1000000.0,assumed_list))
        print("Diff in MB:")
        print(map(lambda x: x/1000000.0,data_usage_diff_list))
        print("Difference in data usages in percent:")
        print(percentage_diff_list)
        print("Time took:")
        print(time_took_list)